#### RAG

The Rag Pipeline consists of Load Data source (Pdf, Media, excel, etc)  ->  Load, Transform and Embed data -> Using query retrieve the data

- Using Chain and Retrival


In [1]:
from langchain_community.document_loaders import PyPDFLoader
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

openai_api_key = os.getenv('OPENAI_API_KEY')

/Users/sanshritsinghai/Desktop/llm/rag_llm/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 5


In [2]:
loader = PyPDFLoader('docs/driver_hos.pdf')
loaded_doc = loader.load()
loaded_doc

Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 71 0 (offset 0)
Ignoring wrong pointing object 90 0 (offset 0)
Ignoring wrong pointing object 124 0 (offset 0)
Ignoring wrong pointing object 147 0 (offset 0)
Ignoring wrong pointing object 148 0 (offset 0)


[Document(metadata={'source': 'docs/driver_hos.pdf', 'page': 0}, page_content='\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\nFederal Motor Carrier Safety Administration \n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\nInterstate \nTruck Driver’s \nGuide  t o \nHours of Service \n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\nOctober 2016'),
 Document(metadata={'source': 'docs/driver_hos.pdf', 'page': 1}, page_content='Interstate Truck Driver’s Guide to Hours of Service \n\t\nTable of Contents \n\t\n\t\nWhat Are the Hours-of-Service Regulations? ............................................................................... 1 \nWho Must Comply With the Hours-of-Service Regulations?........................................................ 1 \nInterstate/Int rastate Commerce ....................................................................................................................  1 \nPersonal Use of a Commercial Motor Vehicle..................

## Data Preprocessing
Text Splitter

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=40)
all_splits = text_splitter.split_documents(loaded_doc)

In [4]:
all_splits

[Document(metadata={'source': 'docs/driver_hos.pdf', 'page': 0}, page_content='Federal Motor Carrier Safety Administration \n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\nInterstate \nTruck Driver’s \nGuide  t o \nHours of Service \n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\nOctober 2016'),
 Document(metadata={'source': 'docs/driver_hos.pdf', 'page': 1}, page_content='Interstate Truck Driver’s Guide to Hours of Service \n\t\nTable of Contents \n\t\n\t\nWhat Are the Hours-of-Service Regulations? ............................................................................... 1 \nWho Must Comply With the Hours-of-Service Regulations?........................................................ 1 \nInterstate/Int rastate Commerce ....................................................................................................................  1'),
 Document(metadata={'source': 'docs/driver_hos.pdf', 'page': 1}, page_content='Personal Use of a Commercial Motor Vehic

Covert these to Embedding - Use OpenAI or Ollama Embedding and using Vectore store to store these embeddings

In [14]:
from langchain_community.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain_community.vectorstores import FAISS
embeddings = OpenAIEmbeddings()
# embeddings = OllamaEmbeddings(model='llama3.2')
db = FAISS.from_documents(loaded_doc,embeddings)


/var/folders/4l/rrftr6v57ls6nj164pgr3w800000gn/T/ipykernel_16154/2265913545.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model='llama3.2')


In [52]:
# query = "How can I plan my investment"
# result = db.similarity_search(query)
# result


[Document(metadata={'source': 'docs/tech_analysis.pdf', 'page': 9}, page_content='Strategy Decision  \nAt this point, there are still many unknowns in how I will approach \nthis small account journey I haven’t made final decisions on brokers \nor software, but I have finalized my trading strategy decision. That is \nwhat I will review here.  \nIf you’re trading a small account yourself, you can download or print \nthis guide to help develop your own strategy.\nWith a small account I have to focus on each trade giving me the biggest return with \nthe least amount of risk. That means my quality threshold is going to be much \nhigher than in my larger trading accounts. When I think about quality, specifically \nthis means I have to make sure that any trade I take meets many of the following \ncriteria- outlined here but covered in more depth below. \n1. The stock makes sense to trade  \n2. I have the right entry point  \n3. I select a reasonable share size  \n4. I manage my risk before an

In [15]:
from langchain_community.llms import Ollama,OpenAI
# llm = Ollama(model= "llama3.2")
llm = OpenAI()
llm


Ollama(model='llama3.2')

### ChatBot

In [16]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the following question strictly only based on the provided document.
                                          <context>
                                          {context}
                                          Question : {input}""")



#### Chains 

Retrival -  It is an interface that returns documents given an unstructured query. More general than vector source.

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain
chain = create_stuff_documents_chain(llm, prompt)
retriver = db.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11fca1190>, search_kwargs={})

#### Retrival Chain
User - (Enquiry)-> Retrieval (Vector Store) -> LLM (through Stuff Document Chain) -> Prompt


In [18]:
### Retrival Chain
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriver, chain)

In [19]:
question = "Generate all possible combination of cases that are possible with respect to Hours of service by a driver. I want it so as to maximize their productivity and utilize their time"
response = retrieval_chain.invoke({"input":question})
response['answer'].split('\n')


['What a complex and fascinating problem!',
 '',
 'To generate all possible combinations of cases for hours of service, we\'ll break down the rules into smaller components and use combinatorics to find all possible scenarios. We\'ll assume that each component can be either "met" or "not met," allowing us to use binary logic.',
 '',
 'Here are the components:',
 '',
 '1. **Driving Time Limit**',
 '\t* Met (M) or Not Met (N)',
 '2. **Sleeper Berth Rule**',
 '\t* 8-9 hours in sleeper berth (M) or Less than 8 hours (N)',
 '3. **Second Rest Period**',
 '\t* 10+ consecutive hours off duty (M) or Less than 10 hours (N)',
 '4. **Electronic Logging Device (ELD)**',
 '\t* Used (U) or Not used (NU)',
 '5. **Work Reporting Location**',
 '\t* Within 100 miles (W) or Outside 100 miles (WO)',
 '6. **Driving Time Limits**',
 '\t* 60-hour/7-day limit (M) or Less than 60 hours/7 days (N)',
 '\t* 70-hour/8-day limit (M) or Less than 70 hours/8 days (N)',
 '',
 "We'll use these components to generate all 